#Load all needed library

In [1]:
!pip install pyspark
import string
import nltk
import re

     |████████████████████████████████| 281.4 MB 30 kB/s 
     |████████████████████████████████| 198 kB 50.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=434001cbd9db1bd2a4dfec053a94fe59d049e7a1124aa511df73d974f9c58591
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("COMP5349 A1") \
    .getOrCreate()

#Initial text processing(same in algorithm1 and algorithm2)

1. Remove (Page xx):
*   In the sentence: re.split("\(Page \d\)")
*   At the end of the sentence: cut the context from the beginning of a sentence to 9 index befor the end of the sentence.(sentence[:-9])

2. Split phrase to keywords:
*   Replace stopwords & punctuations to '|', then split by it.
*   Choose keyword length between 0 to 4 and return.



#Algorithm 1

**step 1**:Using a dictionary to calculate the single word score.

**step 2**:Using another dictionary to calculate keywords socre.

**step 3**:Sorted by keywords score, the top four scores can return (keywords, 1), the other phrase return (keywords, 0).If there are less than four keywords in a phrase, all the element return (keywords, 1). This number means edf value.

result for step3: (keywords, edf)

**step 4 & 5**: Through agreegateByKey() calculate total edf and rdf where the rdf automatically increments by one each time it is traversed.Then using mapValues() calculate essentiality.

Intermediate results(step 4)：(keywords, edf, rdf)

Result for step5: (keywords, essentiality) (essentiality = edf(k)/rdf(k)*edf(k))

**step 6**: Sorted by essentiality and list top 20.

In [44]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('stopwords')
from nltk.corpus import stopwords as sw

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def isPunct(word):
  return len(word) == 1 and word in string.punctuation

def separate_phrase(documents):
  phrase_list = []
  for document in documents:
    sww = sw.words()
    document = map(lambda x: "|" if x in sww else x,
          word_tokenize(document.lower()))
    phrase = []
    for keywords in document:
        if keywords == "|" or isPunct(keywords):  
          if len(phrase) > 0 and len(phrase) < 5:
            phrase_list.append(phrase)
            phrase = []
          elif len(phrase) > 4:
            # phrase_list.append(phrase[0:4])
            phrase = []
        else:
          phrase.append(keywords)
    return phrase_list

def calculate_word_scores(phrase_list):
    word_freq = nltk.FreqDist()
    word_degree = nltk.FreqDist()
    for phrase in phrase_list:
      degree = len(phrase) - 1
      for word in phrase:
        word_freq[word] += 1
        word_degree[word] += degree # other words
    for word in word_freq.keys():
      word_degree[word] = word_degree[word] + word_freq[word] # itself
    # word score = deg(w) / freq(w)
    word_scores = {}
    for word in word_freq.keys():
      word_scores[word] = word_degree[word] / word_freq[word]
    return word_scores

def generate_candidate_keyword_scores(phrase_list, word_score):
  phrase_scores = {}
  for phrase in phrase_list:
    phrase_score = 0
    for word in phrase:
      phrase_score += word_score[word]
    phrase_scores[" ".join(phrase)] = phrase_score
  return phrase_scores

def generate_keyword_edf_pair(keyword_candidates):
  if(len(keyword_candidates)==0):
    return []
  elif(len(keyword_candidates)< 4):
    for phrase in keyword_candidates:
      return [(word, 1) for word in phrase]
  else: 
    keyword_candidates_order =sorted(keyword_candidates.items(),key=lambda x:x[1],reverse=True)
    keyword_candidates_getedf = [(wordScore[0], 1) for wordScore in keyword_candidates_order[:4]]
    keyword_candidates_noedf = [(wordScore[0], 0) for wordScore in keyword_candidates_order[4:]]
    result_candidates = keyword_candidates_getedf + keyword_candidates_noedf
    return [edf_rdf_pair for edf_rdf_pair in result_candidates]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [40]:
def getEdfPairFromGoverningLaw (row):
  row_dict = row.asDict()
  if not row_dict['Governing Law']:
    return []
  else:
    governingLaw = row_dict['Governing Law']
    documents = re.split(pattern="\(Page \d\)", string=governingLaw[:-9].strip())
    phrase_list = separate_phrase(documents)
    word_score = calculate_word_scores(phrase_list)
    keyword_candidates = generate_candidate_keyword_scores(phrase_list, word_score)
    result_candidates = generate_keyword_edf_pair(keyword_candidates)
    return result_candidates

def getEdfPairFromAntiAssignment (row):
  row_dict = row.asDict()
  if not row_dict["Anti-assignment"]:
    return []
  else:
    antiAssignment = row_dict['Anti-assignment'] 
    documents = re.split(pattern="\(Page \d\)", string=antiAssignment[:-9].strip())
    phrase_list = separate_phrase(documents)
    word_score = calculate_word_scores(phrase_list)
    keyword_candidates = generate_candidate_keyword_scores(phrase_list, word_score)
    result_candidates = generate_keyword_edf_pair(keyword_candidates)
    return result_candidates

def getEdfPairFromChangeOfControl (row):
  row_dict = row.asDict()
  if not row_dict['Change of Control']:
    return []
  else:
    changeOfControl = row_dict['Change of Control']
    documents = re.split(pattern="\(Page \d\)", string=changeOfControl[:-9].strip())
    phrase_list = separate_phrase(documents)
    word_score = calculate_word_scores(phrase_list)
    keyword_candidates = generate_candidate_keyword_scores(phrase_list, word_score)
    result_candidates = generate_keyword_edf_pair(keyword_candidates)
    return result_candidates

In [41]:
def seqFunc(accumulatedPair,singleEdf): 
  edf, rdf = accumulatedPair
  edf += singleEdf
  rdf += 1
  return (edf, rdf)
def combFunc(accumulatedPair1,accumulatedPair2):
  edf1, rdf1 = accumulatedPair1
  edf2, rdf2 = accumulatedPair2
  return(edf1 + edf2, rdf1 + rdf2)

def essentiality(value):
  edf, rdf = value
  essentiality = float(edf/rdf)*edf
  return essentiality

In [42]:
rddG = spark.read.csv("Governing_Law.csv",header=True).rdd
edf_keywordsPair_GoverningLaw = rddG.flatMap(getEdfPairFromGoverningLaw)
edf_keywordsPair_GoverningLaw.aggregateByKey((0,0), seqFunc, combFunc, 1)\
              .mapValues(essentiality)\
              .sortBy(lambda x:x[1],ascending=False).take(20)

[('agreement shall', 201.98360655737704),
 ('new york', 53.38888888888889),
 ('without regard', 39.0625),
 ('without giving effect', 37.0),
 ('governed', 36.608450704225355),
 ('agreement', 29.308139534883722),
 ('without reference', 19.36),
 ('construed', 18.0625),
 ('united states', 17.391304347826086),
 ('law principles', 16.025641025641026),
 ('laws principles', 13.78125),
 ('law provisions', 13.473684210526315),
 ('substantive laws', 12.8),
 ('law rules', 11.25),
 ('laws provisions', 9.941176470588236),
 ('laws', 9.651474530831099),
 ('performed entirely within', 9.0),
 ('new york without regard', 9.0),
 ('internal laws', 8.521739130434783),
 ('laws provisions thereof', 8.0)]

In [18]:
rddA = spark.read.csv("Anti_assignment_CIC_g3.csv",header=True).rdd
edf_keywordsPair_Anti = rddA.flatMap(getEdfPairFromAntiAssignment)
edf_keywordsPair_Anti.aggregateByKey((0,0), seqFunc, combFunc, 1)\
              .mapValues(essentiality)\
              .sortBy(lambda x:x[1],ascending=False).take(20)

[('prior written consent', 170.273631840796),
 ('neither party may assign', 48.0),
 ('either party without', 21.77777777777778),
 ('either party may assign', 21.0),
 ('neither party shall assign', 18.0),
 ('agreement may', 17.163265306122447),
 ('a', 13.0),
 ('neither party shall', 13.0),
 ('e', 13.0),
 ('agreement without', 11.571428571428571),
 ('obligations hereunder without', 11.529411764705882),
 ('s', 11.0),
 ('express written consent', 10.0),
 ('party may assign', 9.38888888888889),
 ('t', 9.0),
 ('third party without', 8.642857142857142),
 ('express prior written consent', 8.0),
 ("party 's written consent", 8.0),
 ('prior written approval', 7.111111111111111),
 ('n', 7.0)]

In [43]:
rddA = spark.read.csv("Anti_assignment_CIC_g3.csv",header=True).rdd
edf_keywordsPair_Anti = rddA.flatMap(getEdfPairFromChangeOfControl)
edf_keywordsPair_Anti.aggregateByKey((0,0), seqFunc, combFunc, 1)\
              .mapValues(essentiality)\
              .sortBy(lambda x:x[1],ascending=False).take(20)

[('prior written consent', 7.6923076923076925),
 ('days written notice', 3.0),
 ('licensor may terminate', 3.0),
 ('without limiting', 2.0),
 ('party may assign', 2.0),
 ('shall provide written notice', 2.0),
 ('party may terminate', 2.0),
 ('either party may', 2.0),
 ('either party may terminate', 2.0),
 ('agreement upon written notice', 2.0),
 ('terminated upon', 2.0),
 ('providing prior written notice', 2.0),
 ('licensee upon', 2.0),
 ('substantial portion', 2.0),
 ('`', 2.0),
 ('days prior written notice', 2.0),
 ('provide services shall', 2.0),
 ('provide services', 2.0),
 ('ordinary course', 2.0),
 ('contract period', 2.0)]

#Algorithm 2

**step 1**: extract word with it's keyword

(column 1)   ,    column 2

(('laws', 'shall', 'prevail'), 'laws')

(('laws', 'shall', 'prevail'), 'shall')

(('laws', 'shall', 'prevail'), 'prevail')

(('laws', 'thereof')，'laws')

(('laws', 'thereof'),'thereof')

**step 2**: According to step1 column2 groupBy

('laws', ('laws', 'shall', 'prevail'), ... , ...)

('shall', ('laws', 'shall', 'prevail'),... ,...)

column 2: keywords

**step 3**: According to step2  rdd.map calculate score(degree/frequency)

frequency = len(keywords)

degree = frequency + each keywords lens-1(for example: ('laws', 'shall', 'prevail') = 3-1)

socreMap = ('laws', score)

**step 4**: step1.join(scoreMap).values()

(column 1)   ,    column 2

(('laws', 'shall', 'prevail'), score)

(('laws', 'shall', 'prevail'), score)

(('laws', 'shall', 'prevail'), score)

(('laws', 'thereof')， score)

(('laws', 'thereof'), score)

**step 5**: reduceByKey(x + y)

**step 6**: sortedBy()

In [20]:

def isPunct(word):
  return len(word) == 1 and word in string.punctuation

def replace(documents):
  phrase_list = []
  for document in documents:
    sww = sw.words()
    document = map(lambda x: "|" if x in sww else x,
          word_tokenize(document.lower()))
    phrase = []
    for keywords in document:
        if keywords == "|" or isPunct(keywords):  
          if len(phrase) > 0 and len(phrase) < 5:
            phrase_list.append(phrase)
            phrase = []
          elif len(phrase) > 4:
            # phrase_list.append(phrase[0:4])
            phrase = []
        else:
          phrase.append(keywords)
    return phrase_list

def getWord(phrase_list):
  word_list = []
  for keyword in phrase_list:
    if(len(keyword)< 2):
      word_list.append((''.join(keyword),' '.join(keyword)))
    else:
      for word in keyword:
        word_list.append((''.join(word),' '.join(keyword)))
  return word_list

def countScore(phrase_string_list):
  frequency = len(phrase_string_list[1])
  degree = 0
  for keywords in phrase_string_list:
    degree += len(keywords)
  score = float(degree / frequency)
  return phrase_string_list[0], score

def getAllKeywordsFromGoverningLaw (row):
  row_dict = row.asDict()
  if not row_dict['Governing Law']:
    return []
  else:
    governingLaw = row_dict['Governing Law']
    documents = re.split(pattern="\(Page \d\)", string=governingLaw[:-9].strip())
    phrase_list = replace(documents)
    word_list = getWord(phrase_list)
    return [word for word in word_list]

def getAllKeywordsFromAntiAssignment (row):
  row_dict = row.asDict()
  if not row_dict['Anti-assignment']:
    return []
  else:
    antiAssignment = row_dict['Anti-assignment'] 
    documents = re.split(pattern="\(Page \d\)", string=antiAssignment[:-9].strip())
    phrase_list = replace(documents)
    word_list = getWord(phrase_list)
    return [word for word in word_list]

def getAllKeywordsFromChangeOfControl (row):
  row_dict = row.asDict()
  if not row_dict['Change of Control']:
    return []
  else:
    changeOfControl = row_dict['Change of Control']
    documents = re.split(pattern="\(Page \d\)", string=changeOfControl[:-9].strip())
    phrase_list = replace(documents)
    word_list = getWord(phrase_list)
    return [word for word in word_list]

In [21]:
#Algorithm 2
rddG = spark.read.csv("Governing_Law.csv",header=True).rdd
allKeywordsInGoverningLaw = rddG.flatMap(getAllKeywordsFromGoverningLaw)
scoreMapG = allKeywordsInGoverningLaw.groupBy(lambda x:x[0]).map(countScore)
allKeywordsInGoverningLaw.join(scoreMapG).values().reduceByKey(lambda x,y:x+y).sortBy(lambda x:x[1],ascending=False).take(20)

[('agreement shall', 505.96323215283417),
 ('laws', 460.98858075041005),
 ('state', 396.7115384615371),
 ('governed', 380.7305699481831),
 ('accordance', 302.9659863945569),
 ('construed', 269.4801444043303),
 ('agreement', 196.6723044397472),
 ('new york', 170.90305257207459),
 ('without regard', 135.48807854137434),
 ('without giving effect', 119.47608806963656),
 ('conflict', 118.46218487394957),
 ('conflicts', 116.60176991150452),
 ('law principles', 81.34647887323943),
 ('interpreted', 70.86764705882354),
 ('laws principles', 68.53927808285263),
 ('without reference', 56.431451612903224),
 ('interpretation', 55.36363636363638),
 ('internal laws', 55.26771796266087),
 ('united states', 54.56015037593987),
 ('california', 53.71428571428576)]

In [22]:
rddA = spark.read.csv("Anti_assignment_CIC_g3.csv",header=True).rdd
allKeywordsInAntiAssignment = rddA.flatMap(getAllKeywordsFromAntiAssignment)
scoreMapAnti = allKeywordsInAntiAssignment.groupBy(lambda x:x[0]).map(countScore)
allKeywordsInAntiAssignment.join(scoreMapAnti).values().reduceByKey(lambda x,y:x+y).sortBy(lambda x:x[1],ascending=False).take(20)

[('prior written consent', 684.0612356031706),
 ('agreement', 420.5580357142824),
 ('party', 255.73497267759643),
 ('rights', 251.68987341772046),
 ('neither party may assign', 208.11779440165805),
 ('unreasonably withheld', 183.57647058823545),
 ('assign', 172.96511627906943),
 ('obligations', 158.0579710144929),
 ('transfer', 149.33333333333346),
 ('agreement without', 138.2112607338017),
 ('provided', 122.02597402597368),
 ('third party', 117.3893442622951),
 ('except', 117.37600000000018),
 ('either party without', 113.87417561710944),
 ('obligations hereunder', 113.06235355376216),
 ('assigned', 110.16521739130427),
 ('part', 101.88118811881165),
 ('agreement may', 101.11212073324917),
 ('assignment', 93.5),
 ('without', 92.74043715847003)]

In [23]:
allKeywordsInChangeOfControl = rddA.flatMap(getAllKeywordsFromChangeOfControl)
scoreMapCoC = allKeywordsInAntiAssignment.groupBy(lambda x:x[0]).map(countScore)
allKeywordsInChangeOfControl.join(scoreMapCoC).values().reduceByKey(lambda x,y:x+y).sortBy(lambda x:x[1],ascending=False).take(20)

[('change', 198.66666666666697),
 ('control', 144.6400000000001),
 ('agreement', 137.82142857142878),
 ('occurrence', 102.0),
 ('closing', 88.0),
 ('days', 70.0),
 ('party', 64.4371584699453),
 ('competing product', 60.0),
 ('prior written consent', 58.02305123419765),
 ('circumstances', 56.0),
 ('terminate', 54.0),
 ('commercialization', 54.0),
 ('merger', 49.72972972972972),
 ('indenture trustee', 49.6),
 ('consolidation', 48.5714285714286),
 ('manufacture', 48.0),
 ('event', 46.06060606060604),
 ('additional securities representing', 45.0),
 ('addition', 45.0),
 ('terminated', 43.333333333333336)]